<a href="https://www.kaggle.com/fernandobordi/fb-mapear-covid-centro-arg?scriptVersionId=88414564" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Práctica: mapear COVID en el centro de Argentina con la librería Folium de Python
# Fernando Bordignon - http://saberesdigitales.unipe.edu.ar
 
# Vamos a empezar nuestro programa Python cargando una serie de librerías 
# que nos van a permitir manejar y graficar datos

import pandas as pd      # manejo de estructuras de datos 
import geopandas as gpd  # manejo de datos geográficos
import math              # funciones matemáticas

import folium # para graficar datos geográficos
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

from datetime import datetime, date, time, timedelta # manejo de fechas


/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


Se carga a un **dataframe** el **archivo de datos COVID de las provincias argentinas**: Buenos Aires, La Pampa, Santa Fe, Córdoba, Entre Ríos y el distrito CABA

Atributos:
-ubicacion
-latitud
-longitud
-fecha
-fallecidos
-terapia
-casos_dx
-diagnosticos
-internados
-casos_fis
-casos_fis_ajustada
-casos_fa

Es información diaria acerca del estado de cada provincia. Los datos están en formato csv, almacenados en un espacio de Kaggle. Solamente nos vamos a quedar con una serie de columnas o atributos que nos interesan.

**Consejo:** para aprender a analizar y graficar datos con Python te recomiendo que armes tus propios cuadernos (notebooks) en Kaggle y vayas siguiendo el paso a paso (lo priemro es que gestiones tu cuenta personal). Como siempre, para asistencia extra está Google y las comunidades de usuarios.

**Consejo:** te recomiendo que descargues el archivo cvs a tu computadora y lo explores con una herramienta de planilla de cálculo. De esta manera podrás comprender mejor la estructura del archivo de datos a trabajar. Al archivo lo descargas desde la pestaña "data" en la página principal del presente protecto.

**Tip**: recuerda que en cada cambio que realices o nuevo código que incluyas debes dar "run" a tu proyecto. En principio recomiendo trabajar con "run all"

**Tip**: regularmente graba versiones de tu proyecto con "save verion"


In [2]:
# Carga de datos de un archivo cvs a un dataframe

covid = pd.read_csv("../input/covid19arg/2022-01-06-DsNacion-ARG-centro.csv", encoding='latin-1')

# Los dataframes son estructuras de datos Python provistas por la librería "pandas"
# En principio podemos pensar que un dataframe es como una hoja de calculo, en la cual en
# las filas se almacenan registros de objetos y en las columnas los atributos de cada objeto. 
# En nuestro caso cada fila corrsponde a un registro diario de datos COVID de una provincia

# Se convierte a dato tipo fecha la columna "fecha"

covid['fecha'] = pd.to_datetime(covid['fecha'])

# Se extraen los días del año 2021.  Se aplica un filtro al dataframe "covid"
# El resultado es almacenado en un dataframe llamado "covid" - echas expresadas como mes/día/año

covid = covid[((covid.fecha>="1/1/2021") & (covid.fecha<="12/31/2021"))]

covid.to_csv("2021.csv") # si te interes puede exportar un dataframe a un archivo csv, editable con Excel. Descárgalo de la zona "data"

# Se imprimen las primeras filas del dataframe creado
covid.head()

,ubicacion,latitud,longitud,fecha,fallecidos,terapia,casos_dx,diagnosticos,internados,casos_fis,casos_fis_ajustada,casos_fa
366,buenos aires,-35.002,-61.028,2021-01-01,60,17,1103,3257,81,3494,5184,1292
367,buenos aires,-35.002,-61.028,2021-02-01,77,19,2858,8059,107,3893,4650,2700
368,buenos aires,-35.002,-61.028,2021-03-01,70,16,1694,5483,100,4077,5202,2017
369,buenos aires,-35.002,-61.028,2021-04-01,74,20,4941,13138,137,5571,5602,5596
370,buenos aires,-35.002,-61.028,2021-05-01,74,21,5353,14852,124,5260,4402,5643


Por ejemplo, como vemos en el regitro 366, que corresponde a datos de Buenos Aires del día 01/01/2021, se registraron 60 fallecidos, hay 81 internados, 17 pacientes en terapia intensiva  y los casos nuevos son 1103. Cada registro lleva datos de georeferenciación (latitud y longitud)

Ahora vamos a realizar algunas **operaciones de exploración** del dataframe "covid".

In [3]:
# Ver metadatos del dataframe covid
covid.info()

# Se listan los atributos (columnas) y el tipo de datos que corresponde 
# en cada caso.
# Se observa que hay 2190 registros o filas en el dataframe covid

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2190 entries, 366 to 4415
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ubicacion           2190 non-null   object        
 1   latitud             2190 non-null   float64       
 2   longitud            2190 non-null   float64       
 3   fecha               2190 non-null   datetime64[ns]
 4   fallecidos          2190 non-null   int64         
 5   terapia             2190 non-null   int64         
 6   casos_dx            2190 non-null   int64         
 7   diagnosticos        2190 non-null   int64         
 8   internados          2190 non-null   int64         
 9   casos_fis           2190 non-null   int64         
 10  casos_fis_ajustada  2190 non-null   int64         
 11  casos_fa            2190 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(8), object(1)
memory usage: 222.4+ KB


Vamos a realizar una **operación de filtrado**. Solamente nos vamos a quedar con una serie de columnas o atributos que nos interesan


In [4]:
# Filtrado de columnas que nos interesan

covid = covid[['ubicacion','latitud','longitud','fecha', 
               'fallecidos','terapia', 'diagnosticos', 'internados']]

covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2190 entries, 366 to 4415
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ubicacion     2190 non-null   object        
 1   latitud       2190 non-null   float64       
 2   longitud      2190 non-null   float64       
 3   fecha         2190 non-null   datetime64[ns]
 4   fallecidos    2190 non-null   int64         
 5   terapia       2190 non-null   int64         
 6   diagnosticos  2190 non-null   int64         
 7   internados    2190 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 154.0+ KB


Como vemos en la salida anterior, que muestra **la estructura nueva del dataframe covid**, ahora tenemos solamente nuestros atributos de interés.

**Nota**: fecha es un atributo tipo fecha debido a que anteriormente, usando el método pd.to_datetime, fue convertido a tal tipo.

A continuación vamos a realizar **una operación de suma de valores** de distintos campos. Nos interesan ver los totales de ciertos atributos, para lo cual se usa el método "groupby" aplicado al objeto dataframe "covid". Al agrupar por ubicación tendremos la sumatoria de los valores de cada atributo por cada provincia.

In [5]:
# Queremos saber cuantos registros y atributos válidos tenemos por ubicación

covid.groupby(['ubicacion']).count()

,latitud,longitud,fecha,fallecidos,terapia,diagnosticos,internados
ubicacion,,,,,,,
buenos aires,365,365,365,365,365,365,365
caba,365,365,365,365,365,365,365
cordoba,365,365,365,365,365,365,365
entre rios,365,365,365,365,365,365,365
la pampa,365,365,365,365,365,365,365
santa fe,365,365,365,365,365,365,365


A continuación vamos a **guardar los totales** en un nuevo dataframe llamado covid_totales_2021. Más adelante lo usaremos para graficar en un mapa.

In [6]:
# Nos interesa saber la suma de una serie de atributos por provincia 

covid_totales_2021 = covid.groupby(['ubicacion','latitud','longitud']).sum()
covid_totales_2021 = covid_totales_2021.reset_index() 

# Muestra los primeros registros del dataframe
covid_totales_2021.head()

,ubicacion,latitud,longitud,fallecidos,terapia,diagnosticos,internados
0,buenos aires,-35.002,-61.028,32569,7503,5633954,39775
1,caba,-34.605,-58.480,5568,2805,3550906,12021
2,cordoba,-33.277,-64.269,4498,3831,1856385,12736
3,entre rios,-30.791,-58.810,1750,752,236242,3837
4,la pampa,-36.917,-67.007,878,521,216037,3374


Como vemos en los ejemplos anteriores es fácil calcular y agregar nuevos atributos a los registros.

Ahora, supongamos que nos alcanzan los datos totales de otra provincia, por ejemplo Mendoza, vamos a ver como los **incorporamos al dataframe** covid_totales_2021

In [7]:
# Añadir una fila o registro al dataframe covid_totals_2021

covid_totales_2021 = covid_totales_2021.append({'ubicacion': 'mendoza', 'latitud': -32.880,'longitud': -68.873,
    'fallecidos': 5322, 'terapia': 2133, 'diagnosticos': 2998001, 'internados':10901} , ignore_index=True)

covid_totales_2021

,ubicacion,latitud,longitud,fallecidos,terapia,diagnosticos,internados
0,buenos aires,-35.002,-61.028,32569,7503,5633954,39775
1,caba,-34.605,-58.480,5568,2805,3550906,12021
2,cordoba,-33.277,-64.269,4498,3831,1856385,12736
3,entre rios,-30.791,-58.810,1750,752,236242,3837
4,la pampa,-36.917,-67.007,878,521,216037,3374
5,santa fe,-33.179,-61.569,5211,2346,1010901,7084
6,mendoza,-32.880,-68.873,5322,2133,2998001,10901


In [8]:

# Se van a constuir una serie de indicadores sobre los datos del dataframe covid_totales_2021

# Proporción de internados en terapia sobre total de internados

covid_totales_2021['rel_terapia_internados'] = covid_totales_2021['terapia'] / covid_totales_2021['internados'] 

covid_totales_2021['rel_internados_diags'] = covid_totales_2021['internados'] / covid_totales_2021['diagnosticos'] 

covid_totales_2021['rel_fallecidos_diags'] = covid_totales_2021['fallecidos'] / covid_totales_2021['diagnosticos'] 
    
covid_totales_2021

,ubicacion,latitud,longitud,fallecidos,terapia,diagnosticos,internados,rel_terapia_internados,rel_internados_diags,rel_fallecidos_diags
0,buenos aires,-35.002,-61.028,32569,7503,5633954,39775,0.188636,0.007060,0.005781
1,caba,-34.605,-58.480,5568,2805,3550906,12021,0.233342,0.003385,0.001568
2,cordoba,-33.277,-64.269,4498,3831,1856385,12736,0.300801,0.006861,0.002423
3,entre rios,-30.791,-58.810,1750,752,236242,3837,0.195986,0.016242,0.007408
4,la pampa,-36.917,-67.007,878,521,216037,3374,0.154416,0.015618,0.004064
5,santa fe,-33.179,-61.569,5211,2346,1010901,7084,0.331169,0.007008,0.005155
6,mendoza,-32.880,-68.873,5322,2133,2998001,10901,0.195670,0.003636,0.001775


Como vemos en los ejemplos anteriores es fácil calcular y agregar nuevos atributos a los registros.

**Ahora, a mapear!**

Usando la librería Folium vamos a generar un mapa que nos muestre algunos datos del dataframe covid_totales_2021. Nuestro primer mapa será muy simple, solo queremos saber cuales son las provincias sobre las que tenemo datos.

In [9]:
# Crear un mapa
mapa = folium.Map(location=[-33.68,-63.41], tiles='cartodbpositron', zoom_start=5)

# Se leen los registros del dataframe covid_totales_2021 y se agrega un marcador en el mapa por caada uno
for idx, row in covid_totales_2021.iterrows():
    Marker([row['latitud'], row['longitud']]).add_to(mapa)

# Mostrar el mapa
mapa

Bien, ya está nuestro primer mapa. Es simple, no hay más que marcadores que indiquen provincias sobre las que tenemos información almacenada en el dataframe.

Vamos a hacer un segundo mapa donde **agregamos algunos metadatos** a cada marcador

In [10]:
mapa_con_datos = folium.Map(location=[-33.68,-63.41], zoom_start=6)

for i in range(0,len(covid_totales_2021)):
    marker = folium.Marker(location=[covid_totales_2021.iloc[i]["latitud"], covid_totales_2021.iloc[i]["longitud"]],
        tooltip ="<b>Provincia " +str(covid_totales_2021.iloc[i]["ubicacion"])+"</b> ", 
        popup   ="<b>Fallecidos: "+str(covid_totales_2021.iloc[i]["fallecidos"])+
                "</b></br><b>Internados: "+str(covid_totales_2021.iloc[i]["internados"])+"</b>")
    marker.add_to(mapa_con_datos)

mapa_con_datos.save('mapa.html') # con este método aplicado al mapa lo puedo grabar en un archivo html (revisar la zona output de datos)
mapa

Con el siguiente ejemplo vamos a agregar información visual, por color, al mapa.

In [11]:
ct = covid_totales_2021

# La variable máximo_fallecidos tiene el valor máximo de fallecidos en la tabla covid_totales_2021. Se
# utilizará para calcular luego, por proporciones,el tamaño del marcador en el mapa.
maximo_fallecidos = ct['fallecidos'].max() # Obtengo el máximo de fallecidos en el dataframe covid_totales_2021

mapa_con_color = folium.Map(location=[-33.68,-63.41], zoom_start=6)

for ubica, lat, lon, rel_fall_diags, fallecidos in zip(ct['ubicacion'], ct['latitud'], ct['longitud'], 
    ct['rel_fallecidos_diags'], ct['fallecidos']):
    colorx='#3186cc'
    if (ubica == "buenos aires"): # si la ubicación es Buenos Aires se cambia el color del marcador
        colorx = '#ff0000'
    folium.CircleMarker(
        location=[lat, lon],
        radius= ((fallecidos*100)/maximo_fallecidos)/2, # el radio del marcador se calcula en base a la proporción de fallecidos
        popup=ubica,
        color='#666666',
        fill=True,
        fill_color=colorx,
        fill_opacity=0.4
        ).add_to(mapa_con_color)

mapa_con_color

**Segunda parte del caso**

Ahora crearemos un nuevo dataframe (casos_terapia) sobre el que vamos a generar nuevos mapas

In [12]:
# el dataframe casos_terapia contiene solamente los registros de provincias del día 25/5/2021 donde hay más de 22 
# internados en terapia intensiva

casos_terapia = covid[(covid.terapia > 22) & (covid.fecha=="25/5/2021")]
casos_terapia.head(5)

,ubicacion,latitud,longitud,fecha,fallecidos,terapia,diagnosticos,internados
510,buenos aires,-35.002,-61.028,2021-05-25,282,62,16178,356
1247,caba,-34.605,-58.480,2021-05-25,48,27,8887,103
1984,cordoba,-33.277,-64.269,2021-05-25,38,36,6181,112


Como vemos son tres los registros que cumplen con la condición anterior. Los graficamos en un mapa

In [13]:
# Crear un mapa
mapa = folium.Map(location=[-33.68,-63.41], tiles='cartodbpositron', zoom_start=5)

# Provincias con más de 250 casos en terapia el 25/5/21
for idx, row in casos_terapia.iterrows():
    Marker([row['latitud'], row['longitud']]).add_to(mapa)

# Mostrar el mapa
mapa

Podemos crear otros dataframe auxiliares en base a otras **condiciones**, por ejemplo:

**Dataframe con registros de internados en terapia para mayo de 2021**

In [14]:
# Dataframe con registros de internados en terapia para mayo de 2021 - Las fechas se expresan como mes/dia año
filtro = ( (covid['terapia'] > 0) & ((covid['fecha']>='5/1/2021') & (covid['fecha']<='5/31/2021')) )
terapia_mayo=covid.loc[filtro]

# Dataframe con registros de internados en terapia para mayo de 2021 para CABA (otra forma de filtrar)
caba_terapia_mayo = covid[( (covid.terapia > 0) 
    & ((covid.fecha>="5/1/2021") & (covid.fecha<="5/31/2021")) 
    & (covid.ubicacion=="caba") )]

caba_terapia_mayo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 1107 to 1350
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ubicacion     28 non-null     object        
 1   latitud       28 non-null     float64       
 2   longitud      28 non-null     float64       
 3   fecha         28 non-null     datetime64[ns]
 4   fallecidos    28 non-null     int64         
 5   terapia       28 non-null     int64         
 6   diagnosticos  28 non-null     int64         
 7   internados    28 non-null     int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 2.0+ KB


Vamos a visualizar en un mapa los internados en terapia de mayo 2021. Nótese que el **marcador es tipo cluster** indica cuantos datos agrupa, en nuestro caso representan los días del mes donde hubo pacientes internados en terapia.

In [15]:
# Crear el mapa
mapa_terapia_mayo = folium.Map(location=[-33.68,-63.41], tiles='cartodbpositron', zoom_start=6)

# Agregar marcadores 
markerCluster = folium.plugins.MarkerCluster().add_to(mapa_terapia_mayo)
for index, row in terapia_mayo.iterrows():
    lat=row['latitud']
    lon=row['longitud']
    folium.Marker([lat, lon], popup=row['ubicacion']).add_to(markerCluster)

mapa_terapia_mayo

**Mapa de burbujas**

En un **mapa de burbujas** vamos a representar el **total** de fallecidos del mes de mayo de 2021

In [16]:
fallecidos_mayo = covid[( (covid.fallecidos > 0) 
    & ((covid.fecha>="5/1/2021") & (covid.fecha<="5/31/2021"))  )]

suma_fallecidos_mayo = fallecidos_mayo.groupby(['ubicacion','latitud','longitud']).sum()
suma_fallecidos_mayo = suma_fallecidos_mayo.reset_index()

pd.set_option("max_rows", None)
suma_fallecidos_mayo

,ubicacion,latitud,longitud,fallecidos,terapia,diagnosticos,internados
0,buenos aires,-35.002,-61.028,6000,1365,626216,7530
1,caba,-34.605,-58.480,1024,490,301732,2128
2,cordoba,-33.277,-64.269,760,767,218905,2554
3,entre rios,-30.791,-58.810,362,156,32564,695
4,la pampa,-36.917,-67.007,193,109,30651,783
5,santa fe,-33.179,-61.569,1036,465,126173,1425


In [17]:
# Crear un mapa
mapa_fallecidos_mayo = folium.Map(location=[-33.68,-63.41], tiles='cartodbpositron', zoom_start=6)

def color_producer(val):
    if val <= 1000:
        return 'forestgreen'
    else:
        return 'darkred'

# Agregar las burbujas
for i in range(0,len(suma_fallecidos_mayo)):
    folium.CircleMarker(
        location=[suma_fallecidos_mayo.iloc[i]["latitud"], suma_fallecidos_mayo.iloc[i]["longitud"]],
        radius=25,
        color=color_producer(suma_fallecidos_mayo.iloc[i]["fallecidos"]),
        popup= suma_fallecidos_mayo.iloc[i]["ubicacion"]+" Fallecidos: "+str(suma_fallecidos_mayo.iloc[i]["fallecidos"]),
        fill=True,
    ).add_to(mapa_fallecidos_mayo)

# Mostrar el mapa
mapa_fallecidos_mayo


**Fin del caso de estudio!**, gracias por acompañarnos

Para seguir te recomiendo este [tutorial quickstart](https://python-visualization.github.io/folium/quickstart.html) sobre Folium 

Otros casos de los cuales aprender pueden ser

- [Análisis del crimen en Toronto](https://www.jiristodulka.com/post/toronto-crime/)
- [Estudio de CoVID sobre un cuaderno Google Collab](https://colab.research.google.com/github/pablomartinez/covid_spain/blob/master/covid19.ipynb#scrollTo=cg7QW7COY3Z1)


*Fino a un altro giorno cari amici*

https://www.freecodecamp.org/espanol/news/limpieza-de-datos-en-pandas-explicado-con-ejemplos/

https://medium.com/ironhack/data-cleaning-con-pandas-parte-1-aaa681662b5d
https://medium.com/ironhack/data-cleaning-con-pandas-parte-2-a4e1f55e446b

